<a href="https://colab.research.google.com/github/Anil-matcha/Action_Recognition/blob/master/AR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.layers import LSTM
import numpy as np
import glob,os
#from scipy.misc import imread,imresize
from keras.applications import resnet50
from keras.layers import LSTM

batch_size = 128

resnet_model = resnet50.ResNet50(weights='imagenet')
from keras.layers import Conv2D, Dense, Dropout, TimeDistributed
from keras.models import Model
resnet_model.layers.pop()
resnet_model.layers.pop()
resnet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
from keras.layers.recurrent import GRU
from keras.layers import Reshape, Lambda
from keras import backend as K
from keras.layers.merge import add, concatenate
from keras.layers import Reshape, Lambda, BatchNormalization

inner = Reshape(target_shape=((7, 14336)), name='reshape')(resnet_model.layers[-1].output)  # (None, 32, 2048)
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)
gru_1 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)  # (None, 32, 512)
gru_1b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
reversed_gru_1b = Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_1b)

gru1_merged = add([gru_1, reversed_gru_1b])  # (None, 32, 512)
gru1_merged = BatchNormalization()(gru1_merged)

gru_2 = GRU(256, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(256, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)
reversed_gru_2b= Lambda(lambda inputTensor: K.reverse(inputTensor, axes=1)) (gru_2b)

gru2_merged = concatenate([gru_2, reversed_gru_2b])  # (None, 32, 1024)
gru2_merged = BatchNormalization()(gru2_merged)

out = Dense(7, kernel_initializer='he_normal',name='dense2', activation="softmax")(gru2_merged)

In [33]:
model = Model(input=resnet_model.layers[0].input, output=[out])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/'My Drive'/full_data.zip .
!cp /content/drive/'My Drive'/extras.zip .
!unzip -qq extras.zip
!cp /content/drive/'My Drive'/action_data2.zip .
!unzip -qq action_data2.zip 
!unzip -qq full_data.zip
!mkdir valid
!mkdir valid/burpees
!mkdir valid/mcs
!mkdir valid/squats
!mkdir valid/jj
!mkdir valid/planks
!mkdir valid/pushups
!mv action_data_2/squats/squat2_1_* valid/squats/
!mv action_data_2/jj/jumping_jacks2_1_* valid/jj/
!mv action_data_2/mcs/mountain_climber2_1_* valid/mcs/
!mv action_data_2/planks/plank2_1_* valid/planks/
!mv action_data_2/burpees/burpee2_1_* valid/burpees/
!mv action_data_2/pushups/pushup2_1_* valid/pushups/
!mv action_data_2/squats/* full_data/train/squats/
!mv action_data_2/jj/* full_data/train/jj/
!mv action_data_2/mcs/* full_data/train/mcs/
!mv action_data_2/planks/* full_data/train/planks/
!mv action_data_2/burpees/* full_data/train/burpees/
!mv action_data_2/pushups/* full_data/train/pushups/
!mkdir valid/extras
!mkdir full_data/train/extras
!mv extras/extra_1* valid/extras/
!mv extras/* full_data/train/extras/
!mv valid full_data/

In [0]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        shear_range=0,
        zoom_range=0,
        preprocessing_function = preprocess_input,
        horizontal_flip=False,
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0)
test_datagen = ImageDataGenerator(
        shear_range=0,
        zoom_range=0,
        preprocessing_function = preprocess_input,
        horizontal_flip=False,
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0)

In [35]:
train_generator = train_datagen.flow_from_directory(
    directory=r"./full_data/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=r"./full_data/valid/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 98338 images belonging to 7 classes.
Found 12457 images belonging to 7 classes.


In [0]:
for layer in resnet_model.layers:
  layer.trainable = False

In [0]:
from keras.optimizers import SGD
batch_size = 256
learning_rate = 0.01
sgd = SGD(lr=learning_rate, decay=0.0001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
model.fit_generator(
        train_generator,
        steps_per_epoch=8000 // batch_size,
        epochs=50,
        validation_data=test_generator,
        validation_steps=400 // batch_size)